! Please DONT Run this Notebook Directly... It changes all the Dependencies....

Activate the Environment for BleuRT first

In [1]:
# import torch
# print(torch.__version__)

2.5.1+cu121


In [2]:
# import os
# print(os.getcwd())

# %cd bleurt

/workspace/Approach2/shalaka
/workspace/Approach2/shalaka/bleurt


/workspace/bleurt_env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# !mkdir -p ./bleurt_checkpoints
# !wget -O ./bleurt_checkpoints/base-128.zip https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
    
# !unzip ./bleurt_checkpoints/base-128.zip -d ./bleurt_checkpoints/base-128

--2025-11-25 09:43:37--  https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.192.91, 142.250.70.123, 142.251.222.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.192.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405489453 (387M) [application/zip]
Saving to: ‘./bleurt_checkpoints/base-128.zip’

./bleurt_checkpoint 100%[===================>] 386.70M  15.9MB/s    in 27s     

2025-11-25 09:44:04 (14.3 MB/s) - ‘./bleurt_checkpoints/base-128.zip’ saved [405489453/405489453]

Archive:  ./bleurt_checkpoints/base-128.zip
replace ./bleurt_checkpoints/base-128/bleurt-base-128/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import os
print(os.getcwd())

/workspace/Approach2/shalaka


In [1]:
from bleurt import score
scorer = score.BleurtScorer("./bleurt/bleurt_checkpoints/base-128/bleurt-base-128")

2025-11-25 12:21:50.265753: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 12:21:50.343645: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 12:21:52.650604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


INFO:tensorflow:Reading checkpoint ./bleurt/bleurt_checkpoints/base-128/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


I0000 00:00:1764073318.362018   39247 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18211 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:17:00.0, compute capability: 8.0
I0000 00:00:1764073318.364067   39247 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10229 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0
I0000 00:00:1764073318.365609   39247 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22556 MB memory:  -> device: 2, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:4b:00.0, compute capability: 8.0
I0000 00:00:1764073318.367769   39247 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 26940 MB memory:  -> device: 3, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:ca:00.0, compute capability: 8.0


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [2]:
references = ["This is the test and done."]
candidates = ["This is the test."]

scores = scorer.score(references=references, candidates=candidates)
print(scores)

references = ["This is the test and done.", "abc, d"]
candidates = ["This is the test.", "abcd"]

scores = scorer.score(references=references, candidates=candidates)
print(scores)

[0.25093960762023926]
[0.2506029009819031, -0.22334831953048706]


BLEURT does not have a fixed hard range like BLEU (0–100).
But in practice, BLEURT scores fall roughly between –1.0 and 1.0 for most models.

✅ Practical BLEURT Score Range

–1.0 → Very bad / unrelated translation

0.0 → Poor quality / weak similarity

0.3 – 0.6 → Decent / understandable

0.7 – 0.9 → Very good match

1.0+ → Almost identical (some checkpoints allow >1)

Why no fixed range?

BLEURT is a learned regression model trained to predict human judgments.
Different checkpoints (BLEURT-tiny, BLEURT-base, dBLEURT, BLEURT-20, etc.) produce slightly different ranges.

In [3]:
from bleurt import score
import numpy as np

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEURT scores using all three references for each sentence.

    Args:
        preds: list of predicted sentences
        ref1, ref2, ref3: lists of reference sentences
        bleurt_checkpoint: BLEURT checkpoint to load

    Returns:
        List of BLEURT scores (one score per prediction)
    """
    # Combine references for each sentence
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]
    
    # BLEURT accepts only one reference per prediction at a time.
    # We'll compute the score for each reference and take the max (multi-reference)
    scores = []
    for pred, refs in zip(preds, references):
        ref_scores = [scorer.score(candidates=[pred], references=[ref])[0] for ref in refs]
#         print(pred, " : ", ref_scores)
        scores.append(max(ref_scores))
    
    # Return corpus-level BLEURT as average
    return float(np.mean(scores))

In [4]:
import os
print(os.getcwd())

/workspace/Approach2/shalaka


In [5]:
scorer = score.BleurtScorer("./bleurt/bleurt_checkpoints/base-128/bleurt-base-128")

INFO:tensorflow:Reading checkpoint ./bleurt/bleurt_checkpoints/base-128/bleurt-base-128.


INFO:tensorflow:Reading checkpoint ./bleurt/bleurt_checkpoints/base-128/bleurt-base-128.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint bert_custom


...

INFO:tensorflow:Creating Eager Mode predictor.


INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [6]:
bleurt = {}

## Original

In [7]:
file_name = "shalaka_original_outputs.csv"
mode="original"

In [8]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे आम्ही मागील प्रकल...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [9]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [10]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'original' Model:  0.6238986499331616


## Only Punctuations

In [11]:
file_name = "shalaka_only_punct_outputs.csv"
mode="only_punct"

In [12]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थना केली ते...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी निवासी क्षेत्रात आक्षेपार...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे आम्ही मागील प्रकल...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप भ्रामक असू शकते.","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [13]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [14]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'only_punct' Model:  0.5258963280529888


## Without Punctuations

In [15]:
file_name = "shalaka_without_punct_outputs.csv"
mode="without_punct"

In [16]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या सर्व चारही...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी छोट्या व्यवसायांसाठी पर्य...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,"जसजसे मशीन विकसित होते, तसतसे आपण मागील प्रकल्...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप भ्रामक असू शकते.","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [17]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [18]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'without_punct' Model:  0.5379786728708832


## Combined

In [19]:
file_name = "shalaka_combined_outputs.csv"
mode="combined"

In [20]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा सूर व...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे आपण मागील प्रकल्प...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [21]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [22]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'combined' Model:  0.5348826698544953


## Combined- Shalaka LR=5e-6 Epochs=3

In [23]:
file_name = "shalaka_combined_le_outputs.csv"
mode="combined_le"

In [24]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे मागील प्रकल्पांमध...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [25]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [26]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'combined_le' Model:  0.5703969106630042


## Combined- Shalaka LR=5e-6 Epochs=3 Changed Dataset

In [27]:
file_name = "shalaka_combined_led_outputs.csv"
mode="combined_led"

In [28]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे मागील प्रकल्पांमध...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [29]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [30]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'combined_led' Model:  0.573485629633069


## GPT-5

In [31]:
file_name = "shalaka_gpt_outputs.csv"
mode="gpt"

In [32]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,मंडळातील गायन करताना गायकांचा आवाज वाढवला गेला...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,"सहा महिन्यांचा बछडा तपासणीसाठी सादर केला गेला,...",जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी अशा लघु व्यवसायांसाठी पर्...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,"मशीन जसे स्वरूप विकसित करते, तसेच आपण भूतपूर्व...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","जसे म्हटले, पहिले छाप चुकीचे ठरू शकतात.","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [33]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [34]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'gpt' Model:  0.30104914024748186


## BERT

In [35]:
file_name = "approach1_bert_outputs.csv"
mode="beart"

In [36]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting: the choir raised the volume as the c...,जपः गायकवृंदाने आवाज वाढवला आणि उत्सवी व्यक्ती...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,"As the machine develops, the forms we use to r...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned,, first impressions can be mislea...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [37]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [38]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'beart' Model:  0.6874907267866311


## MPNet

In [39]:
file_name = "approach1_mpnet_outputs.csv"
mode="mpnet"

In [40]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,"Chanting, the choir raised the volume as the c...",उत्सवी व्यक्तीने प्रार्थनेचा उच्चार करत असताना...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-- old calf was submitted for exami...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,"As the machine develops, the forms we use to r...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned,, first impressions can be mislea...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [41]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [42]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 'mpnet' Model:  0.66832000810515


## T5

In [45]:
file_name = "approach1_t5_outputs.csv"
mode="t5"

In [46]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,"Chanting, the choir raised the volume as the c...",उत्सवी व्यक्तीने प्रार्थनेचा उच्चार करत असताना...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,"As the machine develops, the forms we use to r...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [47]:
translations = df['prediction']
ref_gt = df['gt']
ref_gem = df['gemini']
ref_cfilt = df['cfilt']

In [48]:
bleurt_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)
bleurt[mode] = bleurt_score
print(f"BleuRT Score for '{mode}' Model: ", bleurt_score)

BleuRT Score for 't5' Model:  0.6988353191150559


In [49]:
for key, val in bleurt.items():
    print(f"{key} : {val:.4f}")

original : 0.6239
only_punct : 0.5259
without_punct : 0.5380
combined : 0.5349
combined_le : 0.5704
combined_led : 0.5735
gpt : 0.3010
beart : 0.6875
mpnet : 0.6683
t5 : 0.6988
